# Tutorial 7: Custom Protocols

**Goal:** Create pluggable coordination protocols for different control strategies.

**Time:** ~15 minutes

---

## Why Protocols Matter

Protocols define **how coordinators communicate with devices**. HERON's pluggable protocol design lets you:

- **Compare strategies** without changing agent code
- **Experiment** with different coordination mechanisms
- **Publish research** on protocol comparison

### Built-in Protocols

| Protocol | Description | Use Case |
|----------|-------------|----------|
| `SetpointProtocol` | Direct power setpoints | Hierarchical control |
| `PriceSignalProtocol` | Economic signals | Market-based coordination |
| `ConsensusProtocol` | Distributed averaging | Peer-to-peer systems |

In [ ]:
import numpy as np
from typing import Any, Dict

# Import the Protocol base class from HERON
# In practice: from heron.protocols.base import Protocol
# For this tutorial, we define a simplified version
from abc import ABC, abstractmethod

print("Imports ready!")

## Step 1: Understanding the Protocol Interface

A protocol has one core method: `coordinate(coordinator_action, devices) -> device_actions`

In HERON, you can import the base class:
```python
from heron.protocols.base import Protocol
```

In [ ]:
class Protocol(ABC):
    """Base class for coordination protocols.
    
    A protocol translates coordinator-level actions into device-level actions.
    This abstraction allows experimenting with different coordination strategies
    without modifying agent implementations.
    
    Usage with CoordinatorAgent:
        coordinator = PowerGridAgent(
            agent_id='mg_1',
            subordinates=devices,
            protocol=SetpointProtocol(),  # Inject protocol
        )
    """
    
    @abstractmethod
    def coordinate(
        self, 
        coordinator_action: np.ndarray, 
        devices: Dict[str, Any]
    ) -> Dict[str, np.ndarray]:
        """Translate coordinator action to device actions.
        
        Args:
            coordinator_action: Action from coordinator's RL policy
            devices: Dict mapping device_id -> device agent
            
        Returns:
            Dict mapping device_id -> device action
        """
        pass
    
    def reset(self):
        """Reset protocol state (if any)."""
        pass


print("Protocol base class defined!")

## Step 2: SetpointProtocol (Direct Control)

The simplest protocol: coordinator directly sets device power outputs.

In [ ]:
class SetpointProtocol(Protocol):
    """Direct setpoint control.
    
    Coordinator action is a concatenation of device setpoints.
    This protocol simply distributes slices of the action to each device.
    
    Example:
        coordinator_action = [0.5, 0.3, -0.2, 0.8]  # 4 values
        device_1 has action_dim=2 -> gets [0.5, 0.3]
        device_2 has action_dim=2 -> gets [-0.2, 0.8]
    """
    
    def coordinate(
        self, 
        coordinator_action: np.ndarray, 
        devices: Dict[str, Any]
    ) -> Dict[str, np.ndarray]:
        """Distribute action slices to devices."""
        device_actions = {}
        offset = 0
        
        for device_id, device in devices.items():
            action_dim = device.action_space.shape[0]
            device_actions[device_id] = coordinator_action[offset:offset + action_dim]
            offset += action_dim
        
        return device_actions


# Demo
print("SetpointProtocol:")
print("  Coordinator action [0.5, 0.3, -0.2, 0.8]")
print("  -> Device 1: [0.5, 0.3]")
print("  -> Device 2: [-0.2, 0.8]")

## Step 3: PriceSignalProtocol (Market-Based)

Coordinator sends price signals; devices optimize locally based on prices.

In [ ]:
class PriceSignalProtocol(Protocol):
    """Price-based coordination.
    
    Coordinator broadcasts a price signal (or price forecast).
    Devices respond by optimizing their local objective given the price.
    
    This enables:
    - Decentralized decision making
    - Privacy (devices don't share full state)
    - Scalability (no central computation)
    """
    
    def __init__(self, base_price: float = 50.0, price_sensitivity: float = 0.1):
        """Initialize price signal protocol.
        
        Args:
            base_price: Reference price ($/MWh)
            price_sensitivity: How much devices respond to price changes
        """
        self.base_price = base_price
        self.price_sensitivity = price_sensitivity
    
    def coordinate(
        self, 
        coordinator_action: np.ndarray, 
        devices: Dict[str, Any]
    ) -> Dict[str, np.ndarray]:
        """Broadcast price and collect device responses.
        
        Coordinator action is interpreted as price signal(s).
        Devices respond based on their local cost functions.
        """
        # Interpret coordinator action as price signal
        # Action in [-1, 1] maps to price deviation
        price_signal = self.base_price * (1 + coordinator_action[0] * 0.5)
        
        device_actions = {}
        
        for device_id, device in devices.items():
            # Each device responds to price based on its type
            device_action = self._device_response(device_id, device, price_signal)
            device_actions[device_id] = device_action
        
        return device_actions
    
    def _device_response(self, device_id: str, device, price: float) -> np.ndarray:
        """Compute device's optimal response to price signal.
        
        Args:
            device_id: Device identifier (used to infer device type)
            device: Device agent
            price: Current price signal
        """
        action_dim = device.action_space.shape[0]
        
        # Simple heuristic: high price -> reduce consumption, increase generation
        price_deviation = (price - self.base_price) / self.base_price
        
        # Check if device is a generator or storage based on device_id
        if 'generator' in device_id.lower() or 'gen' in device_id.lower():
            # Generator: high price -> increase output
            response = self.price_sensitivity * price_deviation
        elif 'ess' in device_id.lower() or 'battery' in device_id.lower():
            # Storage: high price -> discharge, low price -> charge
            response = self.price_sensitivity * price_deviation
        else:
            response = 0.0
        
        return np.array([np.clip(response, -1, 1)] * action_dim, dtype=np.float32)


print("PriceSignalProtocol:")
print("  Coordinator broadcasts price signal")
print("  Devices optimize locally based on price")
print("  Example: price=75$/MWh -> generators increase output")

## Step 4: ConsensusProtocol (Distributed Averaging)

Devices negotiate to reach agreement without central coordinator.

In [ ]:
class ConsensusProtocol(Protocol):
    """Distributed consensus-based coordination.
    
    Devices iteratively exchange information and converge to
    a consensus through averaging.
    
    This is useful for:
    - Peer-to-peer energy trading
    - Frequency regulation
    - Voltage consensus
    """
    
    def __init__(self, num_iterations: int = 5, mixing_weight: float = 0.5):
        """Initialize consensus protocol.
        
        Args:
            num_iterations: Number of consensus rounds per step
            mixing_weight: Weight for averaging (0-1)
        """
        self.num_iterations = num_iterations
        self.mixing_weight = mixing_weight
        self.device_values = {}  # Track device states
    
    def coordinate(
        self, 
        coordinator_action: np.ndarray, 
        devices: Dict[str, Any]
    ) -> Dict[str, np.ndarray]:
        """Run consensus iterations and return converged actions."""
        
        # Initialize device values from their current states
        device_ids = list(devices.keys())
        n_devices = len(device_ids)
        
        if n_devices == 0:
            return {}
        
        # Initialize with device preferences (based on local state)
        values = {}
        for device_id, device in devices.items():
            # Use device's observation as initial value
            obs = device.observe() if hasattr(device, 'observe') else np.zeros(1)
            values[device_id] = obs[0] if len(obs) > 0 else 0.5
        
        # Target from coordinator (optional guidance)
        target = coordinator_action[0] if len(coordinator_action) > 0 else 0.0
        
        # Run consensus iterations
        for _ in range(self.num_iterations):
            new_values = {}
            avg = np.mean(list(values.values()))
            
            for device_id in device_ids:
                # Mix local value with average and target
                local = values[device_id]
                new_values[device_id] = (
                    (1 - self.mixing_weight) * local + 
                    self.mixing_weight * 0.7 * avg +
                    self.mixing_weight * 0.3 * target
                )
            values = new_values
        
        # Convert consensus values to actions
        device_actions = {}
        for device_id, device in devices.items():
            action_dim = device.action_space.shape[0]
            # Map consensus value to action
            action_value = np.clip(values[device_id], -1, 1)
            device_actions[device_id] = np.full(action_dim, action_value, dtype=np.float32)
        
        return device_actions


print("ConsensusProtocol:")
print("  Devices exchange values iteratively")
print("  Converge to weighted average")
print("  Coordinator provides optional target")

## Step 5: Using Protocols in Agents

Protocols are injected into coordinators at construction time:

```python
from powergrid.agents import PowerGridAgent

# Create coordinator with protocol
microgrid = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=SetpointProtocol(),  # Inject protocol here
)
```

In [ ]:
from gymnasium.spaces import Box

# Simple mock device for demo
class MockDevice:
    def __init__(self, device_id: str, action_dim: int = 1):
        self.device_id = device_id
        self.action_space = Box(-1, 1, (action_dim,), np.float32)
        self._state = np.random.rand()
    
    def observe(self):
        return np.array([self._state])


# Create devices
devices = {
    'gen_1': MockDevice('gen_1', action_dim=2),
    'ess_1': MockDevice('ess_1', action_dim=1),
}

# Test each protocol
coordinator_action = np.array([0.5, 0.3, -0.1])  # 3 values

print("Coordinator action:", coordinator_action)
print()

# SetpointProtocol
setpoint = SetpointProtocol()
actions_setpoint = setpoint.coordinate(coordinator_action, devices)
print("SetpointProtocol output:")
for device_id, action in actions_setpoint.items():
    print(f"  {device_id}: {action}")
print()

# PriceSignalProtocol
price_signal = PriceSignalProtocol(base_price=50.0)
actions_price = price_signal.coordinate(coordinator_action, devices)
print("PriceSignalProtocol output:")
for device_id, action in actions_price.items():
    print(f"  {device_id}: {action}")
print()

# ConsensusProtocol
consensus = ConsensusProtocol(num_iterations=5)
actions_consensus = consensus.coordinate(coordinator_action, devices)
print("ConsensusProtocol output:")
for device_id, action in actions_consensus.items():
    print(f"  {device_id}: {action}")

## Step 6: Swapping Protocols (Key HERON Pattern)

The power of pluggable protocols: **change coordination without touching agent code**.

In [ ]:
class SimpleMicrogrid:
    """Example coordinator that uses pluggable protocols.
    
    This demonstrates the pattern used by PowerGridAgent.
    """
    
    def __init__(self, agent_id: str, protocol: Protocol):
        self.agent_id = agent_id
        self.protocol = protocol  # Injected protocol
        
        # Create subordinate devices
        self.devices = {
            f'{agent_id}_gen': MockDevice(f'{agent_id}_gen', action_dim=2),
            f'{agent_id}_ess': MockDevice(f'{agent_id}_ess', action_dim=1),
        }
    
    def step(self, action: np.ndarray) -> Dict[str, np.ndarray]:
        """Execute step using injected protocol."""
        # Protocol handles action distribution
        device_actions = self.protocol.coordinate(action, self.devices)
        
        # Apply actions to devices
        results = {}
        for device_id, device_action in device_actions.items():
            results[device_id] = device_action
        
        return results


# Same microgrid, different protocols
print("Same agent, different protocols:\n")

action = np.array([0.5, 0.3, -0.1])

# With SetpointProtocol
mg_setpoint = SimpleMicrogrid('mg_1', protocol=SetpointProtocol())
result1 = mg_setpoint.step(action)
print("SetpointProtocol:")
for k, v in result1.items():
    print(f"  {k}: {v}")

print()

# With PriceSignalProtocol
mg_price = SimpleMicrogrid('mg_1', protocol=PriceSignalProtocol())
result2 = mg_price.step(action)
print("PriceSignalProtocol:")
for k, v in result2.items():
    print(f"  {k}: {v}")

print()

# With ConsensusProtocol
mg_consensus = SimpleMicrogrid('mg_1', protocol=ConsensusProtocol())
result3 = mg_consensus.step(action)
print("ConsensusProtocol:")
for k, v in result3.items():
    print(f"  {k}: {v}")

## Step 7: Creating Your Own Protocol

Example: A proportional fairness protocol that ensures fair resource allocation.

In [ ]:
class ProportionalFairnessProtocol(Protocol):
    """Distribute resources proportionally based on device capacities.
    
    Useful when devices have different sizes and you want
    fair allocation based on capacity.
    """
    
    def __init__(self, capacities: Dict[str, float] = None):
        """Initialize with device capacities.
        
        Args:
            capacities: Dict mapping device_id -> capacity
                       If None, assumes equal capacities
        """
        self.capacities = capacities or {}
    
    def coordinate(
        self, 
        coordinator_action: np.ndarray, 
        devices: Dict[str, Any]
    ) -> Dict[str, np.ndarray]:
        """Distribute action proportionally to capacities."""
        # Get capacities (default to 1.0 if not specified)
        capacities = {
            device_id: self.capacities.get(device_id, 1.0)
            for device_id in devices.keys()
        }
        total_capacity = sum(capacities.values())
        
        # Interpret coordinator action as total setpoint
        total_setpoint = coordinator_action[0] if len(coordinator_action) > 0 else 0.0
        
        device_actions = {}
        for device_id, device in devices.items():
            # Proportional share
            share = capacities[device_id] / total_capacity
            action_value = total_setpoint * share
            
            # Fill action vector
            action_dim = device.action_space.shape[0]
            device_actions[device_id] = np.full(action_dim, action_value, dtype=np.float32)
        
        return device_actions


# Test proportional fairness
devices_varied = {
    'large_gen': MockDevice('large_gen', action_dim=1),
    'small_gen': MockDevice('small_gen', action_dim=1),
}

# Large generator has 3x capacity
pf_protocol = ProportionalFairnessProtocol(capacities={
    'large_gen': 3.0,
    'small_gen': 1.0,
})

action = np.array([0.8])  # 80% total setpoint
result = pf_protocol.coordinate(action, devices_varied)

print("ProportionalFairnessProtocol:")
print(f"  Total setpoint: {action[0]}")
print(f"  large_gen (75% capacity): {result['large_gen']}")
print(f"  small_gen (25% capacity): {result['small_gen']}")

## Step 8: Using Protocols with PowerGridAgent

Here's how protocols integrate with the actual PowerGridAgent:

```python
from powergrid.agents import PowerGridAgent, Generator, ESS
from heron.protocols.base import Protocol

# Create devices (FieldAgent level)
devices = {
    'gen_1': Generator(agent_id='gen_1', bus='Bus_1', p_max_MW=2.0),
    'ess_1': ESS(agent_id='ess_1', bus='Bus_1', capacity=5.0),
}

# Create coordinator with protocol (CoordinatorAgent level)
microgrid = PowerGridAgent(
    agent_id='mg_1',
    subordinates=devices,
    protocol=SetpointProtocol(),  # <-- Protocol injection
)

# When coordinator acts, protocol distributes actions to devices:
# coordinator.compute_action() -> protocol.coordinate() -> device actions
```

## Key Takeaways

1. **Protocol Interface**
   ```python
   from heron.protocols.base import Protocol  # In practice
   
   class MyProtocol(Protocol):
       def coordinate(self, coordinator_action, devices) -> Dict[str, np.ndarray]:
           # Transform coordinator action to device actions
           return device_actions
   ```

2. **Built-in Protocols**
   - `SetpointProtocol`: Direct power setpoints (hierarchical)
   - `PriceSignalProtocol`: Economic signals (market-based)
   - `ConsensusProtocol`: Distributed averaging (peer-to-peer)

3. **Pluggable Design**
   ```python
   # Same agent, different coordination
   mg1 = PowerGridAgent('mg_1', subordinates=devs, protocol=SetpointProtocol())
   mg2 = PowerGridAgent('mg_1', subordinates=devs, protocol=PriceSignalProtocol())
   ```

4. **Research Applications**
   - Compare coordination strategies
   - Ablation studies on protocol parameters
   - Hybrid protocols (combine multiple strategies)

---

**Next:** [08_adding_custom_devices.ipynb](08_adding_custom_devices.ipynb) — Add custom device types